# Data join

Programa que sirve para leer y tratar todos los csv ya preprocesados.

- Convierte los csv mensuales que se tienen en local en parquet.
- Se suben manualmente los parquets a Github a la carpeta /data.
- Lee todos los parquets y los agrupa en un solo dataframe para poder trabajar con él.
- Guarda el dataframe en la carpeta data.

### Conversión csv a parquet

In [ ]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from pathlib import Path
import os

folder_path = os.getcwd()
 
csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]

for csv_file in csv_files:
    csv_file = os.path.join(folder_path, csv_file)
    df = pd.read_csv(csv_file)
    
    file_name = os.path.splitext(csv_file)[0]
    parquet_folder = os.path.join(folder_path, f'{file_name}.parquet')
    
    df.to_parquet(parquet_folder)


### Subida a github de los parquets en la carpeta data

Se suben los parquets a github en la carpeta data

### Join de los parquets en un solo df

In [1]:
import os
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

ruta_actual = os.getcwd()
ruta_data = os.path.join(ruta_actual, "data")

archivos_parquet = [archivo for archivo in os.listdir(ruta_data) if archivo.endswith('.parquet')]

dataframes = []

for archivo_parquet in archivos_parquet:
    ruta_parquet = os.path.join(ruta_data, archivo_parquet)
    tabla = pq.read_table(ruta_parquet)
    dataframe = tabla.to_pandas()
    dataframes.append(dataframe)

dataframe_completo = pd.concat(dataframes, ignore_index=True)

In [2]:
#Second post process
dataframe_completo['porcio']=(dataframe_completo['num_docks_available']/dataframe_completo['Llocs'])
dataframe_completo = dataframe_completo.dropna()
dataframe_completo = dataframe_completo.sort_values(['station_id', 'year','houryear'])

dataframe_completo['ctx-1'] = dataframe_completo.groupby(['station_id', 'year'])['porcio'].shift(1)
dataframe_completo['ctx-2'] = dataframe_completo.groupby(['station_id', 'year'])['porcio'].shift(2)
dataframe_completo['ctx-3'] = dataframe_completo.groupby(['station_id', 'year'])['porcio'].shift(3)
dataframe_completo['ctx-4'] = dataframe_completo.groupby(['station_id', 'year'])['porcio'].shift(4)
dataframe_completo = dataframe_completo.dropna()
dataframe_completo = dataframe_completo.dropna()

In [3]:
dataframe_completo

,index,station_id,houryear,num_bikes_available,num_bikes_available_types.mechanical,num_bikes_available_types.ebike,num_docks_available,is_charging_station,is_installed,is_renting,...,Rain_Lectura,year,dayyear,Wind,Wind_Lectura,traffic,ctx-1,ctx-2,ctx-3,ctx-4
752390,691,1,7994,34.000000,31.000000,3.000000,7.000000,1.0,1.0,1.0,...,0.0,2021,334,0,6.2,-5.400000,0.170732,0.170732,0.195122,0.256098
752391,692,1,7995,34.000000,31.000000,3.000000,7.000000,1.0,1.0,1.0,...,0.0,2021,334,0,6.2,-1.050000,0.170732,0.170732,0.170732,0.195122
752392,693,1,7996,34.000000,31.000000,3.000000,7.000000,1.0,1.0,1.0,...,0.0,2021,334,0,6.2,-0.541667,0.170732,0.170732,0.170732,0.170732
752393,694,1,7997,32.916667,29.916667,3.000000,8.083333,1.0,1.0,1.0,...,0.0,2021,334,0,6.2,-1.000000,0.170732,0.170732,0.170732,0.170732
752394,695,1,7998,26.333333,25.416667,0.916667,14.666667,1.0,1.0,1.0,...,0.0,2021,334,0,6.2,-4.800000,0.197154,0.170732,0.170732,0.170732
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
822618,367582,519,8146,0.000000,0.000000,0.000000,22.000000,1.0,1.0,1.0,...,0.0,2021,340,0,7.2,0.683333,1.000000,1.000000,0.954545,0.954545
822619,367583,519,8147,0.000000,0.000000,0.000000,22.000000,1.0,1.0,1.0,...,0.0,2021,340,0,7.2,-0.600000,1.000000,1.000000,1.000000,0.954545
822620,367584,519,8148,0.000000,0.000000,0.000000,22.000000,1.0,1.0,1.0,...,0.0,2021,340,0,7.2,-0.083333,1.000000,1.000000,1.000000,1.000000
822621,367585,519,8149,1.416667,0.000000,1.416667,20.583333,1.0,1.0,1.0,...,0.0,2021,340,0,7.2,0.100000,1.000000,1.000000,1.000000,1.000000


### Se guarda el parquet en la misma carpeta de data

In [4]:
ruta_resultante = os.path.join(ruta_data, "data.parquet")
tabla_resultante = pa.Table.from_pandas(dataframe_completo)
pq.write_table(tabla_resultante, ruta_resultante)